# 데이터 세트에서 임베딩 가져오기

이 노트북은 대규모 데이터 집합에서 임베딩을 가져오는 방법에 대한 예시를 제공합니다.


## 1. 데이터 집합 로드

이 예제에서 사용된 데이터 세트는 Amazon의 [fine-food reviews](https://www.kaggle.com/snap/amazon-fine-food-reviews)입니다. 이 데이터 세트에는 2012년 10월까지 아마존 사용자가 남긴 총 568,454개의 음식 리뷰가 포함되어 있습니다. 예시를 위해 가장 최근 리뷰 1,000개로 구성된 이 데이터 세트의 하위 집합을 사용하겠습니다. 리뷰는 영어로 작성되었으며 긍정적이거나 부정적인 경향이 있습니다. 각 리뷰에는 제품 ID, 사용자 ID, 점수, 리뷰 제목(요약) 및 리뷰 본문(텍스트)이 있습니다.

리뷰 요약과 리뷰 텍스트는 하나의 결합된 텍스트로 결합됩니다. 모델은 이 결합된 텍스트를 인코딩하여 단일 벡터 임베딩을 출력합니다.

이 노트북을 실행하려면 판다, 오픈아이, 트랜스포머, 플로틀리, 매트플롯리브, 스키킷-런, 토치(트랜스포머 디프), 토치비전, 스키피 등을 설치해야 합니다.

In [6]:
# imports
import pandas as pd
import tiktoken

from openai.embeddings_utils import get_embedding


In [7]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191


In [8]:
# load & inspect dataset
input_datapath = "data/fine_food_reviews_1k.csv"  # to save space, we provide a pre-filtered dataset
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)


,Time,ProductId,UserId,Score,Summary,Text,combined
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...


In [9]:
# subsample to 1k most recent reviews and remove samples that are too long
top_n = 1000
df = df.sort_values("Time").tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out
df.drop("Time", axis=1, inplace=True)

encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)


1000

## 2. 임베딩을 받아 나중에 재사용할 수 있도록 저장하세요.

In [10]:
# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage

# This may take a few minutes
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))
df.to_csv("data/fine_food_reviews_with_embeddings_1k.csv")
